In [1]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os

In [2]:
# Only allow png, jpg, and jpeg files
ALLOWED_EXTENSIONS = ['png', 'jpg', 'jpeg']

def allowed_file_types(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [3]:
# Loading and processing the image input
def read_image(filename):
    # Load image
    img = load_img(filename, target_size = (64, 64))

    # Convert to array
    img = img_to_array(img)

    # Reshape for the model
    img = img.reshape(1, 64, 64, 3)

    # Rescale he image
    img = img.astype('float32')
    img = img / 255.

    return img

In [4]:
filename = 'Headshot-me.jpg'

x = read_image(filename)
x.shape

(1, 64, 64, 3)

In [5]:
model = load_model('tuned_nn.h5')
model

In [6]:
model.predict_classes(x)

W1128 15:08:28.311709 140735804920704 deprecation.py:323] From <ipython-input-6-0b3c8f6136df>:1: Sequential.predict_classes (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1]], dtype=int32)

In [7]:
def predict(file):
    try:
        if file and allowed_file_types(file.filename):
            filename = file.filename
            file_path = os.path.join('static/images', filename)
            file.save(file_path)
            img = read_image(file_path)
            
            # Predict the class of an image
            with graph.as_default():
                model1 = load_model('tuned_nn.h5')
                class_prediction = model1.predict_classes(img)
                print(class_prediction)

            #Map apparel category with the numerical class
            if class_prediction[0] == 0:
                 product = "Deepfake Image"
            elif class_prediction[0] == 1: 
                product = "Real Image"

    except Exception:
        return "Unable to read the file. Please make sure that the file extension is correct."
    
    return product

In [8]:
file = 'Headshot-me.jpg'

predict(file)

'Unable to read the file. Please make sure that the file extension is correct.'

In [9]:
def predict_class(file):
    model1 = load_model('tuned_nn.h5')
    pred = model1.predict_classes(file)
    return pred

In [10]:
predict_class(x)

array([[1]], dtype=int32)

In [11]:
file_1 = 'easy_9_1010.jpg'
x_1 = read_image(file_1)
predict_class(x_1)

array([[1]], dtype=int32)

In [12]:
import lime 
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.color import label2rgb

def get_lime_explainer(img, model):
    
    # Get label
    label = predict_class(img)
    
    # Create explainer & Segmenter
    explainer = lime_image.LimeImageExplainer(verbose = False)
    segmenter = SegmentationAlgorithm('slic', n_segments = 100, compactness = 1, sigma = 1)
    
    # Set up explanation
    explanation = explainer.explain_instance(img, classifier_fn = model, top_labels = 2,
                                             hide_color = 0, num_samples = 1000, segmentation_fn = segmenter)
    
    # Plot this thing
    temp, mask = explanation.get_image_and_mask(label, positive_only = True, num_features = 5, 
                                                hide_rest = False)
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8,4))
    ax1.imshow(label2rgb(mask, temp, bg_label = 0), interpolation = 'nearest')
    ax1.set_title('Positve Regions for {}'.format(img))
    temp, mask = explanatio.get_image_and_mask(label, positive_only = False, num_features = 10,
                                               hide_rest = False)
    ax2.imshow(label2rgb(3-mask, temp, bg_label = 0), interpolation = 'nearest')
    ax2.set_title('Positive & Negative Regions for {}'.format(img))

In [13]:
get_lime_explainer(x_1, model)

InvalidArgumentError: input must be 4-dimensional[10,1,62,62,128] [Op:FusedBatchNormV3]

In [29]:
def lime_explainer_image(image_from_test_set, image_label):
    '''Sets up lime explainer and also will display explanation images.
     Image must be in the format of dataset[slice] as in X_test[0].
     Image can be any sliced image from test set.
     Image label is y_test[0] or y_test[1]'''
    
    # Message
    print('This may take a few minutes...')
    
    # Create explainer 
    explainer = lime_image.LimeImageExplainer(verbose = False)
    segmenter = SegmentationAlgorithm('slic', n_segments = 100, compactness = 1, sigma = 1)

    # Set up the explainer
    new_explanation = explainer.explain_instance(image_from_test_set, classifier_fn= model,
                                             top_labels = 2, hide_color = 0, num_samples = 1000, 
                                             segmentation_fn = segmenter)

    # Plot the explainer
    temp, mask = new_explanation.get_image_and_mask(image_label, positive_only=True, num_features=5, hide_rest=False)
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8, 4))
    ax1.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
    ax1.set_title('Positive Regions for {}'.format(image_label))
    temp, mask = new_explanation.get_image_and_mask(image_label, positive_only=False, num_features=10, hide_rest=False)
    ax2.imshow(label2rgb(3-mask,temp, bg_label = 0), interpolation = 'nearest')
    ax2.set_title('Positive/Negative Regions for {}'.format(image_label))

In [32]:
x.shape

(1, 64, 64, 3)

In [34]:
label = model.predict(x).round()
label

array([[1.]], dtype=float32)

In [35]:
lime_explainer_image(x, label)

This may take a few minutes...


InvalidArgumentError: input must be 4-dimensional[10,1,62,62,128] [Op:FusedBatchNormV3]

In [36]:
x

array([[[[0.70980394, 0.45490196, 0.24313726],
         [0.7647059 , 0.5019608 , 0.28627452],
         [0.7647059 , 0.50980395, 0.2901961 ],
         ...,
         [0.7411765 , 0.6784314 , 0.5764706 ],
         [0.7411765 , 0.6784314 , 0.5764706 ],
         [0.7411765 , 0.6784314 , 0.5764706 ]],

        [[0.63529414, 0.4       , 0.20392157],
         [0.6666667 , 0.42352942, 0.22352941],
         [0.68235296, 0.4392157 , 0.23921569],
         ...,
         [0.7490196 , 0.6862745 , 0.58431375],
         [0.7490196 , 0.6862745 , 0.58431375],
         [0.7490196 , 0.6862745 , 0.58431375]],

        [[0.5568628 , 0.33333334, 0.15686275],
         [0.57254905, 0.34117648, 0.16862746],
         [0.59607846, 0.3647059 , 0.19215687],
         ...,
         [0.7529412 , 0.6901961 , 0.5882353 ],
         [0.7529412 , 0.6901961 , 0.5882353 ],
         [0.7529412 , 0.6901961 , 0.5882353 ]],

        ...,

        [[0.2509804 , 0.16862746, 0.08627451],
         [0.21176471, 0.12156863, 0.05098039]

In [37]:
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('slic', n_segments = 100, compactness = 1, sigma = 1)

In [43]:
filename2 = 'fake2.jpg'

file = read_image(filename2)
file.shape

(1, 64, 64, 3)

In [48]:
explanation = explainer.explain_instance(file, classifier_fn = model, top_labels = 4, hide_color = 0, 
                                         num_samples = 1000, segmentation_fn = segmenter)

InvalidArgumentError: input must be 4-dimensional[10,1,62,62,128] [Op:FusedBatchNormV3]

In [49]:
model.predict(file).round()

array([[1.]], dtype=float32)